In [1]:
import pony.orm as po
import numpy as np

In [2]:
db = po.Database()

In [3]:
# A2A    IT0001233417   A2A 255 A2A-S-P-A-Aktie-IT0001233417    EAM.F     FTMIB.MDD    0

class Company(db.Entity):
    isin      = po.PrimaryKey(str)     # company isin
    name      = po.Required(str,unique=True)       # company name
    ticker    = po.Required(str,unique=True)       # ticker symbol
    bench     = po.Set('Benchmark')                # ticker symbol of benchmark
    branch    = po.Set('Branch')                   # branch
    competitor= po.Set('Company')

class Benchmark(db.Entity):
    bench    = po.Required(str)
    
class Web_resources(Company):
    finanzen = po.Required(str)                  # link to finanzen.net
    onvista  = po.Required(str)                  # link to onvista (fundamentals) 
    page     = po.Optional(str)                  # company webpage
    
class Branch(Company):
    bname   = po.Optional(str)
    
class Balance(Company):
#     company  = po.Required(Company)
    year     = po.Required(int) 
    currency = po.Required(str)                  # currency
    # Current Assets
    totass   = po.Optional(float)                # Total Assets
    cash     = po.Optional(float)
    mark_sec = po.Optional(float)                # marketable securities
    acco_rec = po.Optional(float)                # accounts receivable
    note_rec = po.Optional(float)                # notes receivable
    invent   = po.Optional(float)                # inventory
    prep_exp = po.Optional(float)                # prepaid expenses
    # Long Term Assets
    invest   = po.Optional(float)                # investments
    fixe_ass = po.Optional(float)                # fixed assets
    othe_ass = po.Optional(float)                # other assets
    inta_ass = po.Optional(float)                # intangiable assets
    # Current Liabilities
    curr_lia = po.Optional(float)                # current liabilities
    long_lia = po.Optional(float)                # long term liabilities
    totlia   = po.Optional(float)                # total liabilities
    
class Fundamentals(Company):
#     company  = po.Required(Company)
#     balance  = po.Required(balance)              # balance
    year     = po.Required(int)
    EPS      = po.Optional(float)                # earnings per share
    PER      = po.Optional(float)                # price earnings ratio
    dividend = po.Optional(float)                # dividend
    divyield = po.Optional(float)                # dividend yield
    CPS      = po.Optional(float)                # cashflow per share
    
class Income_Statement(Company):
#     company  = po.Required(Company)
    year     = po.Required(int)       
    currency = po.Required(str)
    #
    tot_rev  = po.Optional(float)                # total revenues
    tot_cost = po.Optional(float)                # total cost and expenses
    inc_oper = po.Optional(float)                # income from operations
    pro_inct = po.Optional(float)                # provision for income taxes
    net_incn = po.Optional(float)                # net income per share basic
    net_incd = po.Optional(float)                # net income per share diluted
    
    

ERDiagramError: Attribute Fundamentals.year conflicts with attribute Balance.year because both entities inherit from Company. To fix this, move attribute definition to base class

In [9]:
# db.bind('sqlite', ':memory:')
# pwd
db.bind('sqlite', '/home/phermes/Dropbox/codes/python/170102_stocks_sql/stockdb.sqlite', create_db=True)

In [7]:
pwd

u'/home/phermes/Dropbox/codes/python/170102_stocks_sql'

In [10]:
db.generate_mapping(create_tables=True)

In [11]:
data = np.genfromtxt('stocks.lst',dtype='str')
webp = np.genfromtxt('webpages.lst',dtype='str')

In [12]:
for [name, isin, finanzen, non, onvista, tick, btick, branch] in data:
    
    c1 = Company(
            isin     =  isin,
            name     =  name,
            ticker   =  tick,
            bench    =  btick,
            branch   =  branch
                )
    
    try:
        w = webp[webp[:,0]==isin][0][1]
    except:
        w = 'http://www.google.com'
        
    w1 = Web_resources(
            company  = c1,
            onvista  = onvista,
            finanzen = finanzen,
            page     = w,
            )
    
db.commit()

In [12]:
po.select(c for c in Web_resources).order_by(Web_resources.id)[:100].show()

id |company     |finanzen            |onvista             |page                
---+------------+--------------------+--------------------+--------------------
1  |Company[1]  |A2A                 |A2A-S-P-A-Aktie-I...|http://www.a2a.eu...
2  |Company[2]  |Aareal_Bank         |Aareal-Bank-Aktie...|http://www.aareal...
3  |Company[3]  |ABB_1               |ABB-LTD-Aktie-US0...|http://www.abb.com  
4  |Company[4]  |ABB                 |ABB-Aktie-CH00122...|http://www.abb.com  
5  |Company[5]  |Abbott_Laboratories |ABBOTT-LABORATORI...|http://www.abbott...
6  |Company[6]  |Abengoa             |ABENGOA-S-A-Aktie...|http://www.abengo...
7  |Company[7]  |AbercrombieFitch    |ABERCROMBIE-Aktie...|http://www.abercr...
8  |Company[8]  |Aberdeen_Asset_Ma...|ABERDEEN-ASSET-MA...|http://www.aberde...
9  |Company[9]  |Abertis_Infraestr...|ABERTIS-INFRAESTR...|http://www.aberti...
10 |Company[10] |ABIOMED             |ABIOMED-Aktie-US0...|http://www.abiome...
11 |Company[11] |ACADIA_Pharmaceut...|AC

In [11]:
po.select(c for c in Company).order_by(Company.id)[:100].show()

id |isin        |name                               |ticker |bench    |branch
---+------------+-----------------------------------+-------+---------+------
1  |IT0001233417|A2A                                |EAM.F  |FTMIB.MDD|0     
2  |DE0005408116|Aareal_Bank                        |ARL.DE |^GDAXI   |1     
3  |US0003752047|ABB_1                              |ABJA.DE|^GSPC    |0     
4  |CH0012221716|ABB                                |ABJ.DE |^SSMI    |0     
5  |US0028241000|Abbott_Laboratories                |ABL.DE |^GSPC    |0     
6  |ES0105200416|Abengoa                            |AYO.DE |^IBEX    |0     
7  |US0028962076|AbercrombieFitch                   |AFT.F  |^GSPC    |0     
8  |GB0000031285|Aberdeen_Asset_Management          |324.F  |^FTSE    |0     
9  |ES0111845014|Abertis_Infraestructuras           |AUC.F  |^IBEX    |0     
10 |US0036541003|ABIOMED                            |AIO.F  |^GSPC    |0     
11 |US0042251084|ACADIA_Pharmaceuticals             |DR6.F  |^GS